In [1]:
# set run environment (local/colab)
# if colab install required packages and set appropriate root_path
import os

if os.getenv("COLAB_RELEASE_TAG"):
    colab = True
    from google.colab import drive
    drive.mount('/content/drive')
    root_path = '/content/drive/Othercomputers/My computer/EQILLM/'
    !pip install -r '/content/drive/Othercomputers/My computer/EQILLM/requirements.txt'
    !pip install transformers[torch]
    !pip install accelerate -U
    !pip install datasets
    !pip install torchinfo
    import sys
    sys.path.append(root_path)
    #ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U
else:
    colab = False
    root_path = ''


import itertools
import pandas as pd
import openai
import datetime
import os
import csv

from dotenv import load_dotenv, dotenv_values
from tqdm.notebook import tqdm_notebook
from eqillm import finetune, get_log_for_val, validate, val_metrics, yeelight_eow_notification, param_combinations, load_PolarIs, df_to_ds


dotenv_config = dotenv_values(os.path.join(root_path, '.env'))
yeelight_notify = dotenv_config['YEELIGHT_NOTIFY'] if 'YEELIGHT_NOTIFY' in dotenv_config else False

In [2]:
# from huggingface_hub import notebook_login
# notebook_login()

# models = ['albert', 'bart', 'bert', 'big_bird', 'bigbird_pegasus', 'biogpt', 'bloom', 'camembert', 'canine', 'code_llama', 'convbert', 'ctrl', 'data2vec-text', 'deberta', 'deberta-v2', 'distilbert', 'electra', 'ernie', 'ernie_m', 'esm', 'falcon', 'flaubert', 'fnet', 'funnel', 'gemma', 'gpt-sw3', 'gpt2', 'gpt_bigcode', 'gpt_neo', 'gpt_neox', 'gptj', 'ibert', 'layoutlm', 'layoutlmv2', 'layoutlmv3', 'led', 'lilt', 'llama', 'longformer', 'luke', 'markuplm', 'mbart', 'mega', 'megatron-bert', 'mistral', 'mixtral', 'mobilebert', 'mpnet', 'mpt', 'mra', 'mt5', 'mvp', 'nezha', 'nystromformer', 'open-llama', 'openai-gpt', 'opt', 'perceiver', 'persimmon', 'phi', 'plbart', 'qdqbert', 'qwen2', 'reformer', 'rembert', 'roberta', 'roberta-prelayernorm', 'roc_bert', 'roformer', 'squeezebert', 'stablelm', 'starcoder2', 't5', 'tapas', 'transfo-xl', 'umt5', 'xlm', 'xlm-roberta', 'xlm-roberta-xl', 'xlnet', 'xmod', 'yoso',]
# =============================================
# works = [']
# ----------
#too weak pc (cannot be loaded or runs at speeds <1 it/s) = 'albert-xlarge-v1', 't5-11B', 'LongformerForSequenceClassification', 'xlm-roberta-base', 'allenai/longformer-base-4096', 'facebook/bart-large',  flaubert/flaubert_large_cased (stopped at 0.5 epoch),
# ----------
# need changes to run (check error logs) =  ProsusAI/finbert (despite adding padding proposed in error)
# =============================================
# 'xlnet-base-cased' - needs data collator

# , 'camembert-base', 'ctrl', 'xlnet-base-cased', 'roberta-base', 'distilroberta-base', 'flaubert/flaubert_base_cased'

# deprecated: transfo-xl-wt103


# ['michellejieli/emotion_text_classifier', 'xlnet-base-cased', 'roberta-base', 'distilroberta-base', 'flaubert/flaubert_base_cased']



# to_solve


In [3]:
#if list of objects is provided list of all combinations of parameters will be created for running
#['michellejieli/emotion_text_classifier', 'xlnet-base-cased', 'roberta-base', 'distilroberta-base', 'flaubert/flaubert_base_cased']

params_tested = {'model': [
                            # 'mistralai/Mistral-7B-Instruct-v0.2',
                           # 'cardiffnlp/twitter-roberta-base-irony',
                           # 'nlptown/bert-base-multilingual-uncased-sentiment',
                           # 'michellejieli/emotion_text_classifier',
                           # 'j-hartmann/emotion-english-distilroberta-base',
                           # 'distilroberta-base',
                           # 'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           # 'ProsusAI/finbert',
                           # 'cardiffnlp/twitter-roberta-base-sentiment-latest',
                           # 'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
                           # 'joeddav/distilbert-base-uncased-go-emotions-student',
                           # 'camembert-base'
                           # 'joeddav/distilbert-base-uncased-go-emotions-student',
                           # 'ctrl',
                           # 'camembert-base',
                           # 'papluca/xlm-roberta-base-language-detection',
                           # 'mistralai/Mistral-7B-v0.1',
                           # 'cardiffnlp/twitter-roberta-base-irony',
                           # 'meta-llama/Meta-Llama-3-8B',
                           # 'distilbert/distilbert-base-uncased-finetuned-sst-2-english', 
                           # 'j-hartmann/emotion-english-distilroberta-base', 
                           # 'arpanghoshal/EmoRoBERTa', 
                           # 'ProsusAI/finbert', 
                           # 'cardiffnlp/twitter-roberta-base-sentiment-latest', 
                           # 'michellejieli/emotion_text_classifier', 
                           # 'xlnet-base-cased', 
                           # 'roberta-base', 
                           # 'distilroberta-base', 
                           # 'flaubert/flaubert_base_cased',
                           #  'celine98/canine-s-finetuned-sst2',
                           #  'lytang/MiniCheck-Flan-T5-Large',
                           #  'michelecafagna26/t5-base-finetuned-sst2-sentiment',
                           #  'facebook/tart-full-flan-t5-xl',
                           #  'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           #  'nlptown/bert-base-multilingual-uncased-sentiment',
                           #  'cardiffnlp/twitter-xlm-roberta-base-sentiment'
                           ], # Pre-trained model names from the Hugging Face hub used for fine-tuning
                 'num_train_epochs': 10, # Number of times the model sees the entire training dataset.
                 'save_strategy': 'no', # Controls when to save model checkpoints ('epoch' or 'no').
                 'per_device_train_batch_size': 8, # Number of samples processed in each training step (personally, 8/16 work best, 16 is faster, but you may find linear drop in inference speed during fine-tuning).
                 'per_device_eval_batch_size': 64, # Number of samples processed in each evaluation step.
                 'split': (0.6, 0.2, 0.2), # Divides the dataset into training, testing, (and optionally) validation sets. Use 'balanced' for equal class representation in the validation set. Examples: (90,10) -> split into train and test proportionally; (80, 10, 10) splits into train,test, validate proportionally.
                 'binary': [True], # Indicates whether the task is binary (two classes) or multi-class classification.,
                 'balanced': [False], # his way labels used for training are split evenly, fitting size to the lowest label count. n (equal to 80% of least represented label) will be taken from each label, rest will be used for test.
                 }

# Controls whether to save logs during a process. When set to False, logging is disabled.
save_logs = True

train_params_looped = param_combinations(params_tested)
print(len(train_params_looped))
print(*train_params_looped, sep='\n')

1
{'model': 'mistralai/Mistral-7B-Instruct-v0.2', 'num_train_epochs': 10, 'save_strategy': 'no', 'per_device_train_batch_size': 8, 'per_device_eval_batch_size': 64, 'split': (0.6, 0.2, 0.2), 'binary': True, 'balanced': False}


In [4]:
data_path = os.path.join(root_path, 'data/PolarIs-Pathos.xlsx')


#run looped
for train_params in tqdm_notebook(train_params_looped):
    polarIs_df = load_PolarIs(data_path)
    dataset, target_map, reversed_target_map = df_to_ds(polarIs_df, train_params['split'], train_params['binary'], train_params['balanced'])
    finetune(dataset, train_params, target_map, reversed_target_map, save_logs, root_path, colab)

if yeelight_notify:
    yeelight_eow_notification(dotenv_config['YEELIGHT_PORT'])

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/12470 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1559 [00:00<?, ? examples/s]

Map:   0%|          | 0/1559 [00:00<?, ? examples/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Jakub\DataspellProjects\EQILLM\venv\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Error: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.
